In [1]:
# importing libraries
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import contractions
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,roc_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import scikitplot as skplt
import seaborn as sns
from sklearn import metrics


In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Importing the data

In [2]:
import json
data = [json.loads(line) for 
        line in open("Sarcasm_Headlines_Dataset.json", 'r')]

In [3]:
new_df = pd.DataFrame.from_dict(data) 

In [4]:
data = new_df.drop(['article_link'],axis=1)

In [5]:
data = data.head(100)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      100 non-null    object
 1   is_sarcastic  100 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [9]:
data.head(2)

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0


# Splitting the data

In [11]:
x_train,x_test,y_train,y_test = train_test_split(data.headline,data.is_sarcastic,test_size=0.25,random_state=42)

# Preprocessing functions

In [12]:
# creating functions for preprocessing
def remove_newlines(data):
    formatted_text = data.replace("\\n",' ').replace("\t"," ")
    return formatted_text

def contraction_map(data):
    fixed_text = contractions.fix(data)
    return fixed_text

def handle_accented(data):
    fixed_text = unidecode(data)
    return fixed_text

stopword_list = stopwords.words("english")
stopword_list.remove("no")
stopword_list.remove("not")
stopword_list.remove("nor")
def cleaning_text(data):
    tokens = word_tokenize(data)
    clean_tokens = [ i.lower() for i in tokens if (i.lower() not in stopword_list) and (i not in punctuation) ]
    clean_tokens = [ i for i in  clean_tokens if (len(i)>1) and i.isalpha()]
    return clean_tokens

def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    final_tokens = []
    for i in data:
        lemmatized_word = lemmatizer.lemmatize(i)
        final_tokens.append(lemmatized_word)
    return " ".join(final_tokens)

In [13]:
clean_train = x_train.apply(remove_newlines)
clean_test = x_test.apply(remove_newlines)

clean_train = clean_train.apply(contraction_map)
clean_test = clean_test.apply(contraction_map)

clean_train = clean_train.apply(handle_accented)
clean_test = clean_test.apply(handle_accented)

clean_train = clean_train.apply(cleaning_text)
clean_test = clean_test.apply(cleaning_text)

clean_train = clean_train.apply(lemmatization)
clean_test = clean_test.apply(lemmatization)

# word indexing

In [15]:
# word indexing
max_words = 1000
tk = Tokenizer(num_words=max_words,oov_token="##oov##")
train_sent_list = clean_train.to_list()
test_sent_list = clean_test.to_list()
tk.fit_on_texts(train_sent_list)
x_train_seq = tk.texts_to_sequences(train_sent_list)
x_test_seq = tk.texts_to_sequences(test_sent_list)

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_words_per_sent = 500
x_train_seq = pad_sequences(x_train_seq,padding='post',maxlen = max_words_per_sent,truncating='post')
x_test_seq = pad_sequences(x_test_seq,padding='post',maxlen = max_words_per_sent,truncating='post')

# Building simple RNN

In [18]:
from tensorflow.keras.layers import Dense,Embedding,SimpleRNN,Bidirectional,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [19]:
model = Sequential()
embdding_dim = 50
model.add(Embedding(input_dim = max_words+1,output_dim = embdding_dim,input_length = max_words_per_sent))
model.add(SimpleRNN(units=64,return_sequences=True))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(x_train_seq,y_train,batch_size=252,epochs=10,validation_data=(x_test_seq,y_test),callbacks=[es])

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6890 - accuracy: 0.5470 - val_loss: 0.6017 - val_accuracy: 0.7570
Epoch 2/10
1/1 [==============================] - 0s 251ms/step - loss: 0.6978 - accuracy: 0.5474 - val_loss: 0.6347 - val_accuracy: 0.7569
Epoch 3/10
1/1 [==============================] - 0s 256ms/step - loss: 0.6890 - accuracy: 0.5479 - val_loss: 0.6633 - val_accuracy: 0.7323
Epoch 4/10
1/1 [==============================] - 0s 245ms/step - loss: 0.6861 - accuracy: 0.5491 - val_loss: 0.6337 - val_accuracy: 0.7458


# Deep RNN

In [20]:
model = Sequential()
embdding_dim = 50
model.add(Embedding(input_dim = max_words+1,output_dim = embdding_dim,input_length = max_words_per_sent))
model.add(SimpleRNN(units=64,return_sequences=True))
model.add(SimpleRNN(units=32,return_sequences=True))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(x_train_seq,y_train,batch_size=252,epochs=10,validation_data=(x_test_seq,y_test),callbacks=[es])

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.7075 - accuracy: 0.4634 - val_loss: 0.6240 - val_accuracy: 0.7474
Epoch 2/10
1/1 [==============================] - 0s 327ms/step - loss: 0.6928 - accuracy: 0.5425 - val_loss: 0.6825 - val_accuracy: 0.7437
Epoch 3/10
1/1 [==============================] - 0s 314ms/step - loss: 0.6934 - accuracy: 0.5385 - val_loss: 0.6530 - val_accuracy: 0.7481
Epoch 4/10
1/1 [==============================] - 0s 328ms/step - loss: 0.6899 - accuracy: 0.5415 - val_loss: 0.6356 - val_accuracy: 0.7487


# Bidirectional RNN

In [21]:
model = Sequential()
embdding_dim = 50
model.add(Embedding(input_dim = max_words+1,output_dim = embdding_dim,input_length = max_words_per_sent))
model.add(Bidirectional(SimpleRNN(units=64,return_sequences=True)))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(x_train_seq,y_train,batch_size=252,epochs=10,validation_data=(x_test_seq,y_test),callbacks=[es])


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.6944 - accuracy: 0.4644 - val_loss: 0.6117 - val_accuracy: 0.7594
Epoch 2/10
1/1 [==============================] - 0s 327ms/step - loss: 0.6942 - accuracy: 0.5466 - val_loss: 0.6487 - val_accuracy: 0.7582
Epoch 3/10
1/1 [==============================] - 0s 392ms/step - loss: 0.6887 - accuracy: 0.5467 - val_loss: 0.6557 - val_accuracy: 0.7594
Epoch 4/10
1/1 [==============================] - 0s 384ms/step - loss: 0.6888 - accuracy: 0.5470 - val_loss: 0.6516 - val_accuracy: 0.7597


# LSTM

In [22]:
model = Sequential()
embdding_dim = 50
model.add(Embedding(input_dim = max_words+1,output_dim = embdding_dim,input_length = max_words_per_sent))
model.add(LSTM(units=8,return_sequences=True))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(x_train_seq,y_train,batch_size=252,epochs=10,validation_data=(x_test_seq,y_test),callbacks=[es])

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.6934 - accuracy: 0.4542 - val_loss: 0.6929 - val_accuracy: 0.7530
Epoch 2/10
1/1 [==============================] - 0s 222ms/step - loss: 0.6931 - accuracy: 0.5463 - val_loss: 0.6917 - val_accuracy: 0.7588
Epoch 3/10
1/1 [==============================] - 0s 218ms/step - loss: 0.6929 - accuracy: 0.5473 - val_loss: 0.6902 - val_accuracy: 0.7600
Epoch 4/10
1/1 [==============================] - 0s 257ms/step - loss: 0.6926 - accuracy: 0.5469 - val_loss: 0.6886 - val_accuracy: 0.7600
Epoch 5/10
1/1 [==============================] - 0s 290ms/step - loss: 0.6923 - accuracy: 0.5468 - val_loss: 0.6867 - val_accuracy: 0.7600
Epoch 6/10
1/1 [==============================] - 0s 244ms/step - loss: 0.6920 - accuracy: 0.5467 - val_loss: 0.6847 - val_accuracy: 0.7600
Epoch 7/10
1/1 [==============================] - 0s 237ms/step - loss: 0.6917 - accuracy: 0.5467 - val_loss: 0.6825 - val_accuracy: 0.7600
Epoch 8/10
1/1 [=======

# Bidirectional LSTM

In [23]:
model = Sequential()
embdding_dim = 50
model.add(Embedding(input_dim = max_words+1,output_dim = embdding_dim,input_length = max_words_per_sent))
model.add(Bidirectional(LSTM(units=8,return_sequences=True)))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(x_train_seq,y_train,batch_size=252,epochs=10,validation_data=(x_test_seq,y_test),callbacks=[es])


Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 0.6926 - accuracy: 0.5457 - val_loss: 0.6860 - val_accuracy: 0.7592
Epoch 2/10
1/1 [==============================] - 0s 334ms/step - loss: 0.6919 - accuracy: 0.5469 - val_loss: 0.6823 - val_accuracy: 0.7597
Epoch 3/10
1/1 [==============================] - 0s 310ms/step - loss: 0.6914 - accuracy: 0.5471 - val_loss: 0.6786 - val_accuracy: 0.7598
Epoch 4/10
1/1 [==============================] - 0s 332ms/step - loss: 0.6908 - accuracy: 0.5472 - val_loss: 0.6745 - val_accuracy: 0.7600
Epoch 5/10
1/1 [==============================] - 0s 318ms/step - loss: 0.6903 - accuracy: 0.5471 - val_loss: 0.6703 - val_accuracy: 0.7600
Epoch 6/10
1/1 [==============================] - 0s 343ms/step - loss: 0.6898 - accuracy: 0.5470 - val_loss: 0.6660 - val_accuracy: 0.7600
Epoch 7/10
1/1 [==============================] - 0s 324ms/step - loss: 0.6894 - accuracy: 0.5469 - val_loss: 0.6617 - val_accuracy: 0.7600
Epoch 8/10
1/1 [=======